## 0. Prepare the Virtual Environment

In [ ]:
!pip install --q virtualenv

In [ ]:
%cd "/content/drive/MyDrive/4. Semester 4/final"

/content/drive/MyDrive/4. Semester 4/final


In [ ]:
!python3 -m virtualenv venv

created virtual environment CPython3.10.12.final.0-64 in 8815ms
  creator CPython3Posix(dest=/content/drive/MyDrive/4. Semester 4/final/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.2, setuptools==75.2.0, wheel==0.44.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
!source venv/bin/activate

In [ ]:
import sys
sys.path.insert(0, "/content/drive/MyDrive/4. Semester 4/final/venv/lib/python3.8/site-packages")

In [ ]:
!pip install -r requirements.txt

## 1. Ingest PDF for training data

In [ ]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 582 kB/3,744 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ub

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [ ]:
data_path = "/content/drive/MyDrive/4. Semester 4/final/IATA_2013.pdf"

### 1.1. Table detection and extraction

In [ ]:
!pip install camelot-py[cv]
!pip install PyPDF2==2.10.0
!apt-get install -y ghostscript

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.9).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import camelot

tables = camelot.read_pdf(data_path, pages='all')

/usr/local/lib/python3.10/dist-packages/camelot/parsers/lattice.py:397: UserWarning: No tables found on page-2
  warnings.warn(


In [ ]:
def flatten_table(table_df):
    flattened = ""
    headers = [str(header) for header in table_df.columns.tolist()]
    flattened += " | ".join(headers) + "\n"
    flattened += "-" * 20 + "\n"
    for row in table_df.iterrows():
        flattened += " | ".join([str(cell) for cell in row[1].tolist()]) + "\n"
    return flattened


In [ ]:
!pip install pymupdf

In [ ]:
import fitz

doc = fitz.open(data_path)
non_table_text = ""
for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    text = page.get_text("text")
    non_table_text += text

Remove unwanted elements and merge tables and non-tables contents into cleaned text

In [ ]:
import re
cleaned_non_table_text = re.sub(r'[^\x20-\x7E]', '', non_table_text)
cleaned_non_table_text = re.sub(r'\s+', ' ', cleaned_non_table_text).strip()

cleaned_text = cleaned_non_table_text
for i, table in enumerate(tables):
    cleaned_text += f"\n\nFlattened Table {i + 1}:\n"
    cleaned_text += flatten_table(table.df)

In [ ]:
cleaned_text_path = "/content/drive/MyDrive/4. Semester 4/final/cleaned_text.txt"
with open(cleaned_text_path, "w") as text_file:
    text_file.write(cleaned_text)

Extract a new pdf file for data

In [ ]:
!pip install reportlab

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def save_text_as_pdf(text, pdf_path):
    c = canvas.Canvas(pdf_path, pagesize=letter)
    width, height = letter
    c.drawString(72, height - 72, text)
    c.save()

pdf_output_path = "/content/drive/MyDrive/4. Semester 4/final/cleaned_text.pdf"
save_text_as_pdf(cleaned_text, pdf_output_path)
print(f"Cleaned text saved as PDF to: {pdf_output_path}")

Cleaned text saved as PDF to: /content/drive/MyDrive/4. Semester 4/final/cleaned_text.pdf


### 1.2. Load the cleaned text

In [ ]:
!pip install --upgrade nltk

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
loader = UnstructuredPDFLoader(file_path=pdf_output_path)
data = loader.load()

## 2. Vector Embeddings

In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13320    0 13320    0     0  32426      0 --:--:-- --:--:-- --:--:-- 32408>>> Installing ollama to /usr/local
100 13320    0 13320    0     0  31455      0 --:--:-- --:--:-- --:--:-- 31415
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama pull llama3.2

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 34bb5ab01051... 100% ▕▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
!ollama list

NAME                       ID              SIZE      MODIFIED       
llama3.2:latest            a80c4f17acd5    2.0 GB    45 minutes ago    
nomic-embed-text:latest    0a109f422b47    274 MB    53 minutes ago    


In [ ]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

### 2.1. Split the data into smaller chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [ ]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="llama3.2",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 486/486 [06:22<00:00,  1.27it/s]


## 3. Build a Retriever

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
llm = ChatOllama(model="llama3.2")

In [ ]:
PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant designed to retrieve the most relevant and accurate information from a large database.
    Your task is to process the user’s question, retrieve precise data from the vector database, and generate an accurate and coherent answer.
    Carefully analyze the user question and identify the key information needed.
    Retrieve the most relevant data from the vector database that aligns with the question.
    Ensure that the data retrieved is factually correct and provides comprehensive coverage of the topic.
    If multiple pieces of information are found, combine them logically to formulate a clear and accurate response.
    Always verify that the response is well-structured, concise, and provides the most correct version of the answer.

    Original question: {question}""",
)

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=PROMPT
)

template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    | (lambda x: x.replace("\n", " "))
    | (lambda x: x.replace("\t", " "))
)

In [ ]:
chain.invoke(input(""))

what are the guidelines for Chemicals and solvents


OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 16.56it/s]


"Based on the provided text, here are the guidelines for chemicals and solvents:  **General Guidelines**  * Flammable solids (Flamm. solid): Keep away from heat. * Explosives (Explosive 1.4): Follow specific regulations and guidelines.  **Specific Chemicals and Solvents**  * Signal devices, hand: (UN 0191) * Fireworks: (UN 0333-0337) * Shaped charges: (UN 0059, 0439, 0440, 0441) * Shellac: (Paint, UN 1263) * Paint: (Shellac, UN 1263)  **Classification and Packaging**  * Based on the classification (e.g. Flamm. solid, Explosive 1.4) and packaging quantities:  + 5 kg or less: Forbidden  + 10 kg or more: Forbidden  **Storage and Handling**  * Keep away from heat for flammable solids. * Follow specific regulations and guidelines for explosives.  **Special Considerations**  * Distress signals (Signals, distress) * Hand signals (Signal devices, hand) * Water-activated signals (Contrivances, water-activated)  Note that these guidelines are not exhaustive and may vary depending on the specific

In [ ]:
vector_db.delete_collection()